# Keras tuner test

In [ ]:
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.6 MB/s eta 0:00:00


In [ ]:
import keras_tuner

https://keras.io/guides/keras_tuner/getting_started/

https://keras.io/guides/keras_tuner/visualize_tuning/

## Phase 1

In [ ]:
def build_model(hp):

    model_name = hp.Choice('Model', ['VGG19','EfficientNetV2B0','MobileNetV2'])

    if model_name == 'VGG19':
      base_model = keras.applications.VGG19(
          weights="imagenet",  # Load weights pre-trained on ImageNet.
          input_shape=(image_size[0], image_size[1], 3),
          include_top=False,
      )  # Do not include the ImageNet classifier at the top.

    elif model_name == 'EfficientNetV2B0':
      base_model = keras.applications.EfficientNetV2B0(
          weights="imagenet",  # Load weights pre-trained on ImageNet.
          input_shape=(image_size[0], image_size[1], 3),
          include_top=False,
          include_preprocessing=False,
      )  # Do not include the ImageNet classifier at the top.

    elif model_name == 'MobileNetV2':
      base_model = keras.applications.MobileNetV2(
          weights="imagenet",  # Load weights pre-trained on ImageNet.
          input_shape=(image_size[0], image_size[1], 3),
          include_top=False,
          alpha=1.0
      )  # Do not include the ImageNet classifier at the top.

    # Freeze the base_model
    base_model.trainable = False

    # Create new model on top
    inputs = keras.Input(shape=(image_size[0], image_size[1], 3))


    # Pre-trained Xception weights requires that input be scaled
    # from (0, 255) to a range of (-1., +1.), the rescaling layer
    # outputs: `(inputs * scale) + offset`
    scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
    x = scale_layer(inputs)

    # The base model contains batchnorm layers. We want to keep them in inference mode
    # when we unfreeze the base model for fine-tuning, so we make sure that the
    # base_model is running in inference mode here.
    x = base_model(x, training=False)

    top_layers = hp.Choice('Top layers', ['GlobalAveragePooling2D','FlattenAndDense','Flatten'])

    if top_layers == 'GlobalAveragePooling2D':
      x = keras.layers.GlobalAveragePooling2D()(x)
      x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout

    elif top_layers == 'FlattenAndDense':
      x = keras.layers.Flatten()(x)
      x = keras.layers.Dense(
          units=256,
          activation='relu'
      )(x)

      x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout

    elif top_layers == 'Flatten':
      x = keras.layers.Flatten()(x)

    outputs = keras.layers.Dense(
        units=8,
        activation='softmax'
    )(x)

    model = keras.Model(inputs, outputs)

    model.summary(show_trainable=True)

    model.compile(
    optimizer=keras.optimizers.RMSprop(), # optimizer,
    loss=keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy'],
    )
    return model

In [ ]:
build_model(keras_tuner.HyperParameters())

80134624/80134624 [==============================] - 4s 0us/step
Model: "model"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         Y          
                                                                            
 rescaling (Rescaling)       (None, 150, 150, 3)       0         Y          
                                                                            
 vgg19 (Functional)          (None, 4, 4, 512)         2002438   N          
                                                       4                    
                                                                            
 global_average_pooling2d (  (None, 512)               0         Y          
 GlobalAveragePooling2D)                                                    
                                                                         

In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    max_trials=9,
    executions_per_trial=1,
    # Do not resume the previous search in the same directory.
    overwrite=True,
    objective="val_accuracy",
    # Set a directory to store the intermediate results.
    directory="/tmp/phase_1-1",
    project_name="CRC classification",
)


Model: "model_1"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_4 (InputLayer)        [(None, 150, 150, 3)]     0         Y          
                                                                            
 rescaling_1 (Rescaling)     (None, 150, 150, 3)       0         Y          
                                                                            
 vgg19 (Functional)          (None, 4, 4, 512)         2002438   N          
                                                       4                    
                                                                            
 global_average_pooling2d_1  (None, 512)               0         Y          
  (GlobalAveragePooling2D)                                                  
                                                                            
 dropout_1 (Dropout)         (None, 512)               0   

In [ ]:
tuner.search(
    train_ds,
    epochs=10,
    validation_data=validation_ds,
    # Use the TensorBoard callback.
    # The logs will be write to "/tmp/phase_1-1".
    callbacks=[keras.callbacks.TensorBoard("/tmp/phase_1-1")],
)

Trial 9 Complete [00h 01m 31s]
val_accuracy: 0.8979057669639587

Best val_accuracy So Far: 0.9123036861419678
Total elapsed time: 00h 17m 30s


In [ ]:
! zip -r ./phase_1-1.zip /tmp/phase_1-1

In [ ]:
! pwd

/content


In [ ]:
%load_ext tensorboard

%tensorboard --logdir /tmp/phase_1-1

<IPython.core.display.Javascript object>

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
Model (Choice)
{'default': 'VGG19', 'conditions': [], 'values': ['VGG19', 'EfficientNetV2B0', 'MobileNetV2'], 'ordered': False}
Top layers (Choice)
{'default': 'GlobalAveragePooling2D', 'conditions': [], 'values': ['GlobalAveragePooling2D', 'FlattenAndDense', 'Flatten'], 'ordered': False}


In [ ]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
best_model.summary()

Model: "model"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         Y          
                                                                            
 rescaling (Rescaling)       (None, 150, 150, 3)       0         Y          
                                                                            
 efficientnetv2-b0 (Functio  (None, 5, 5, 1280)        5919312   N          
 nal)                                                                       
                                                                            
 global_average_pooling2d (  (None, 1280)              0         Y          
 GlobalAveragePooling2D)                                                    
                                                                            
 dropout (Dropout)           (None, 1280)              0     

In [ ]:
tuner.results_summary()

Results summary
Results in /tmp/phase_1-1/CRC classification
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
Model: EfficientNetV2B0
Top layers: GlobalAveragePooling2D
Score: 0.9123036861419678

Trial 8 summary
Hyperparameters:
Model: EfficientNetV2B0
Top layers: FlattenAndDense
Score: 0.8979057669639587

Trial 6 summary
Hyperparameters:
Model: MobileNetV2
Top layers: FlattenAndDense
Score: 0.8939790725708008

Trial 3 summary
Hyperparameters:
Model: EfficientNetV2B0
Top layers: Flatten
Score: 0.8913612365722656

Trial 4 summary
Hyperparameters:
Model: MobileNetV2
Top layers: GlobalAveragePooling2D
Score: 0.8848167657852173

Trial 7 summary
Hyperparameters:
Model: MobileNetV2
Top layers: Flatten
Score: 0.8782722353935242

Trial 2 summary
Hyperparameters:
Model: VGG19
Top layers: FlattenAndDense
Score: 0.8704188466072083

Trial 5 summary
Hyperparameters:
Model: VGG19
Top layers: Flatten
Score: 0.8612565398216248

Trial 0 summary
Hy

## Phase 2

### EfficientNetV2B0

In [ ]:
def build_model(hp):

    base_model = keras.applications.EfficientNetV2B0(
        weights="imagenet",  # Load weights pre-trained on ImageNet.
        input_shape=(image_size[0], image_size[1], 3),
        include_top=False,
        include_preprocessing=False,
    )  # Do not include the ImageNet classifier at the top.

    # Freeze the base_model
    base_model.trainable = False

    # Create new model on top
    inputs = keras.Input(shape=(image_size[0], image_size[1], 3))

    # Pre-trained Xception weights requires that input be scaled
    # from (0, 255) to a range of (-1., +1.), the rescaling layer
    # outputs: `(inputs * scale) + offset`
    scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
    x = scale_layer(inputs)

    # The base model contains batchnorm layers. We want to keep them in inference mode
    # when we unfreeze the base model for fine-tuning, so we make sure that the
    # base_model is running in inference mode here.
    x = base_model(x, training=False)

    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
    outputs = keras.layers.Dense(
        units=8,
        activation='softmax',
    )(x)

    model = keras.Model(inputs, outputs)
    model.summary(show_trainable=True)

    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    optimizer_name = hp.Choice('optimizer', ['Adam','RMSprop', 'Adagrad', 'Nadam', 'NAG'])

    if optimizer_name == 'Adam':
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_name == 'RMSprop':
        optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif optimizer_name == 'Adagrad':
        optimizer = keras.optimizers.Adagrad(learning_rate=learning_rate)
    elif optimizer_name == 'Nadam':
        optimizer = keras.optimizers.Nadam(learning_rate=learning_rate)
    elif optimizer_name == 'NAG':
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)

    model.compile(
      optimizer=optimizer,
      loss=keras.losses.sparse_categorical_crossentropy,
      metrics=['accuracy'],
    )
    return model

In [ ]:
tuner = keras_tuner.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='tuner_res',
                     project_name='efficientnetv2-b0_phase1')

24274472/24274472 [==============================] - 2s 0us/step
Model: "model"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         Y          
                                                                            
 rescaling (Rescaling)       (None, 150, 150, 3)       0         Y          
                                                                            
 efficientnetv2-b0 (Functio  (None, 5, 5, 1280)        5919312   N          
 nal)                                                                       
                                                                            
 global_average_pooling2d (  (None, 1280)              0         Y          
 GlobalAveragePooling2D)                                                    
                                                                         

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner.search(train_ds, epochs=10, validation_data=validation_ds, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 20 Complete [00h 00m 48s]
val_accuracy: 0.8913612365722656

Best val_accuracy So Far: 0.9070680737495422
Total elapsed time: 00h 12m 29s


In [ ]:
tuner.results_summary()

Results summary
Results in tuner_res/efficientnetv2-b0_phase1
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0016 summary
Hyperparameters:
learning_rate: 0.001
optimizer: Nadam
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0014
Score: 0.9070680737495422

Trial 0017 summary
Hyperparameters:
learning_rate: 0.001
optimizer: Adam
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0015
Score: 0.9044502377510071

Trial 0011 summary
Hyperparameters:
learning_rate: 0.01
optimizer: Adam
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.8939790725708008

Trial 0019 summary
Hyperparameters:
learning_rate: 0.01
optimizer: Nadam
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.8913612365722656

Trial 0014 summary
Hyperparameters:
learning_rate: 0.001
optimizer: Nadam
tuner/epochs: 4
tuner/initial_epoch: 2
tuner/bracket: 2
tuner/ro

In [ ]:
best_hps.values

{'learning_rate': 0.001,
 'optimizer': 'Nadam',
 'tuner/epochs': 10,
 'tuner/initial_epoch': 4,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0014'}

In [ ]:
! zip -r ./efficientnetv2-b0_phase1.zip ./tuner_res/efficientnetv2-b0_phase1

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 20 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_ds, epochs=20, validation_data=validation_ds)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch))


Model: "model_3"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_8 (InputLayer)        [(None, 150, 150, 3)]     0         Y          
                                                                            
 rescaling_3 (Rescaling)     (None, 150, 150, 3)       0         Y          
                                                                            
 efficientnetv2-b0 (Functio  (None, 5, 5, 1280)        5919312   N          
 nal)                                                                       
                                                                            
 global_average_pooling2d_3  (None, 1280)              0         Y          
  (GlobalAveragePooling2D)                                                  
                                                                            
 dropout_3 (Dropout)         (None, 1280)              0   

Train model again with the best params and number of epochs

In [ ]:
base_model = keras.applications.EfficientNetV2B0(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(image_size[0], image_size[1], 3),
    include_top=False,
    include_preprocessing=False,
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(image_size[0], image_size[1], 3))

# Pre-trained Xception weights requires that input be scaled
# from (0, 255) to a range of (-1., +1.), the rescaling layer
# outputs: `(inputs * scale) + offset`
scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(inputs)

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)

x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(
    units=8,
    activation='softmax',
)(x)

model = keras.Model(inputs, outputs)
model.summary(show_trainable=True)

model.compile(
    optimizer=keras.optimizers.Nadam(learning_rate=0.001),
    loss=keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy'],
)

print("Fitting the top layer of the model")
model.fit(train_ds, epochs=15, validation_data=validation_ds)

24274472/24274472 [==============================] - 0s 0us/step
Model: "model"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         Y          
                                                                            
 rescaling (Rescaling)       (None, 150, 150, 3)       0         Y          
                                                                            
 efficientnetv2-b0 (Functio  (None, 5, 5, 1280)        5919312   N          
 nal)                                                                       
                                                                            
 global_average_pooling2d (  (None, 1280)              0         Y          
 GlobalAveragePooling2D)                                                    
                                                                         

In [ ]:
eval_result = model.evaluate(test_ds)
print("[test loss, test accuracy]:", eval_result)

23/23 [==============================] - 1s 35ms/step - loss: 0.2331 - accuracy: 0.9266
[test loss, test accuracy]: [0.23314984142780304, 0.926630437374115]


In [ ]:
model.save(f"{os.environ['DRIVE_DATA_DIR']}/EfficientNetV2B0-Nadam-0.001.keras")

### VGG19

## Phase 3

### EfficientNetV2B0

In [ ]:
model.summary(show_trainable=True)

Model: "model"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         Y          
                                                                            
 rescaling (Rescaling)       (None, 150, 150, 3)       0         Y          
                                                                            
 efficientnetv2-b0 (Functio  (None, 5, 5, 1280)        5919312   N          
 nal)                                                                       
                                                                            
 global_average_pooling2d (  (None, 1280)              0         Y          
 GlobalAveragePooling2D)                                                    
                                                                            
 dropout (Dropout)           (None, 1280)              0     

In [ ]:
# Unfreeze the base_model. Note that it keeps running in inference mode
# since we passed `training=False` when calling it. This means that
# the batchnorm layers will not update their batch statistics.
# This prevents the batchnorm layers from undoing all the training
# we've done so far.
base_model.trainable = True
model.summary(show_trainable=True)

Model: "model"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         Y          
                                                                            
 rescaling (Rescaling)       (None, 150, 150, 3)       0         Y          
                                                                            
 efficientnetv2-b0 (Functio  (None, 5, 5, 1280)        5919312   Y          
 nal)                                                                       
                                                                            
 global_average_pooling2d (  (None, 1280)              0         Y          
 GlobalAveragePooling2D)                                                    
                                                                            
 dropout (Dropout)           (None, 1280)              0     

In [ ]:
def build_model(hp):

    model_copy = keras.models.clone_model(model)
    model_copy.set_weights(model.get_weights())

    learning_rate = hp.Choice('learning_rate', values=[1e-3, 5e-4, 1e-4])
    optimizer_name = hp.Choice('optimizer', ['Adam','RMSprop', 'Adagrad', 'Nadam', 'NAG'])

    if optimizer_name == 'Adam':
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_name == 'RMSprop':
        optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif optimizer_name == 'Adagrad':
        optimizer = keras.optimizers.Adagrad(learning_rate=learning_rate)
    elif optimizer_name == 'Nadam':
        optimizer = keras.optimizers.Nadam(learning_rate=learning_rate)
    elif optimizer_name == 'NAG':
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)

    model_copy.compile(
      optimizer=optimizer,
      loss=keras.losses.sparse_categorical_crossentropy,
      metrics=['accuracy'],
    )

    return model

In [ ]:
tuner = keras_tuner.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='tuner_res',
                     project_name='efficientnetv2-b0_phase2')

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner.search(train_ds, epochs=10, validation_data=validation_ds, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 20 Complete [00h 00m 31s]
val_accuracy: 0.9175392389297485

Best val_accuracy So Far: 0.9201570749282837
Total elapsed time: 00h 09m 09s


In [ ]:
tuner.results_summary()

Results summary
Results in tuner_res/efficientnetv2-b0_phase2
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0017 summary
Hyperparameters:
learning_rate: 0.0001
optimizer: Nadam
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0014
Score: 0.9201570749282837

Trial 0007 summary
Hyperparameters:
learning_rate: 0.0001
optimizer: RMSprop
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.9188481569290161

Trial 0011 summary
Hyperparameters:
learning_rate: 0.0005
optimizer: Adagrad
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.9188481569290161

Trial 0016 summary
Hyperparameters:
learning_rate: 0.0001
optimizer: RMSprop
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0012
Score: 0.9188481569290161

Trial 0018 summary
Hyperparameters:
learning_rate: 0.0005
optimizer: NAG
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracke

In [ ]:
best_hps.values

{'learning_rate': 0.0001,
 'optimizer': 'Nadam',
 'tuner/epochs': 10,
 'tuner/initial_epoch': 4,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0014'}

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 20 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_ds, epochs=20, validation_data=validation_ds)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch))


Epoch 1/20
110/110 [==============================] - 5s 48ms/step - loss: 0.0955 - accuracy: 0.9697 - val_loss: 0.2574 - val_accuracy: 0.9175
Epoch 2/20
110/110 [==============================] - 7s 58ms/step - loss: 0.0967 - accuracy: 0.9691 - val_loss: 0.2590 - val_accuracy: 0.9123
Epoch 3/20
110/110 [==============================] - 7s 59ms/step - loss: 0.0984 - accuracy: 0.9674 - val_loss: 0.2600 - val_accuracy: 0.9149
Epoch 4/20
110/110 [==============================] - 7s 63ms/step - loss: 0.0988 - accuracy: 0.9680 - val_loss: 0.2556 - val_accuracy: 0.9175
Epoch 5/20
110/110 [==============================] - 7s 58ms/step - loss: 0.0944 - accuracy: 0.9714 - val_loss: 0.2570 - val_accuracy: 0.9162
Epoch 6/20
110/110 [==============================] - 7s 60ms/step - loss: 0.0879 - accuracy: 0.9743 - val_loss: 0.2568 - val_accuracy: 0.9162
Epoch 7/20
110/110 [==============================] - 5s 46ms/step - loss: 0.0888 - accuracy: 0.9709 - val_loss: 0.2602 - val_accuracy: 0.9110

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(train_ds, epochs=best_epoch, validation_data=validation_ds)

Epoch 1/9
110/110 [==============================] - 11s 96ms/step - loss: 0.0844 - accuracy: 0.9714 - val_loss: 0.2645 - val_accuracy: 0.9162
Epoch 2/9
110/110 [==============================] - 7s 64ms/step - loss: 0.0823 - accuracy: 0.9749 - val_loss: 0.2721 - val_accuracy: 0.9084
Epoch 3/9
110/110 [==============================] - 5s 46ms/step - loss: 0.0800 - accuracy: 0.9769 - val_loss: 0.2737 - val_accuracy: 0.9097
Epoch 4/9
110/110 [==============================] - 7s 62ms/step - loss: 0.0786 - accuracy: 0.9751 - val_loss: 0.2696 - val_accuracy: 0.9097
Epoch 5/9
110/110 [==============================] - 7s 60ms/step - loss: 0.0744 - accuracy: 0.9780 - val_loss: 0.2798 - val_accuracy: 0.9097
Epoch 6/9
110/110 [==============================] - 9s 79ms/step - loss: 0.0831 - accuracy: 0.9709 - val_loss: 0.2708 - val_accuracy: 0.9110
Epoch 7/9
110/110 [==============================] - 15s 135ms/step - loss: 0.0756 - accuracy: 0.9769 - val_loss: 0.2651 - val_accuracy: 0.9123
Epo

In [ ]:
eval_result = hypermodel.evaluate(test_ds)
print("[test loss, test accuracy]:", eval_result)

23/23 [==============================] - 1s 36ms/step - loss: 0.2150 - accuracy: 0.9253
[test loss, test accuracy]: [0.21495695412158966, 0.92527174949646]


### VGG19